# Dataset Trackingについて

MLFLowに対して、データセットに関する情報も記録可能である。

ここで用いる主な抽象コンポーネントとして、`Dataset` と `DatasetSource`の2つがある。

### Dataset

`Dataset`は、特定のログされたデータセットに関する情報を保持するメタデータトラッキングオブジェクトである。

`Dataset`オブジェクトに保存される情報には、特徴量、ターゲット、予測、およびデータセットの名前、ダイジェスト（ハッシュ）、スキーマ、プロファイルなどのメタデータが含まれる。  
このメタデータは `mlflow.log_input()` APIを使用してログできる。  
モジュールは、さまざまなデータタイプから `mlflow.data.dataset.Dataset` オブジェクトを構築するための関数を提供する。

この抽象クラスの具体的な実装には以下が含まれます：

* `mlflow.data.spark_dataset.SparkDataset`
* `mlflow.data.pandas_dataset.PandasDataset`
* `mlflow.data.numpy_dataset.NumpyDataset`
* `mlflow.data.huggingface_dataset.HuggingFaceDataset`
* `mlflow.data.tensorflow_dataset.TensorFlowDataset`

### DatasetSource

`DatasetSource`は、与えられた`Dataset`オブジェクトのコンポーネントであり、データの元のソースへのリンクされた系統を提供する。

`Dataset`の`DatasetSource`コンポーネントは、データセットのソースを表し、S3のディレクトリ、デルタテーブル、またはURLなどが該当する。  
これは、データの起源を理解するために`Dataset`内で参照される。  
ログされたデータセットの`DatasetSource`は、`Dataset`オブジェクトの`source`プロパティにアクセスするか、`mlflow.data.get_source()` APIを使用することで取得できます。

MLflow内でサポートされている多くの自動ログ有効化フレーバーは、データセット自体をログする際に自動的にデータセットのソースをロギングを行う。

## 検証

MLFlowのGithubリポジトリにあるwinequality-whiteデータセットを用いて、Dataset Trackingの挙動について確認する。

まずはライブラリのインポートを行い、サーバにMLFlow Sessionを接続する。

In [1]:
import mlflow
import pandas as pd
from mlflow.data.pandas_dataset import PandasDataset

In [2]:
mlflow.set_tracking_uri("http://localhost:5000")

実験名も以下の通り定義する。

In [3]:
mlflow.set_experiment("Dataset-Tracking")

2024/04/24 16:01:27 INFO mlflow.tracking.fluent: Experiment with name 'Dataset-Tracking' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/902688651372085152', creation_time=1713942087202, experiment_id='902688651372085152', last_update_time=1713942087202, lifecycle_stage='active', name='Dataset-Tracking', tags={}>

### データセットに関する処理

MLFlowのGithubリポジトリにあるwinequality-whiteデータセットをダウンロードし、Trackingを行う。  
ダウンロード用のURLを`dataset_source_url`という変数に格納する。

In [4]:
dataset_source_url = "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv"
dataset_source_url

'https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv'

`dataset_source_url`にあるcsvファイルのデータを、データフレーム`df`に取り込む。

In [5]:
df = pd.read_csv(dataset_source_url, delimiter=";")
df

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.00100,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.99400,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.99510,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.99560,3.19,0.40,9.9,6
...,...,...,...,...,...,...,...,...,...,...,...,...
4893,6.2,0.21,0.29,1.6,0.039,24.0,92.0,0.99114,3.27,0.50,11.2,6
4894,6.6,0.32,0.36,8.0,0.047,57.0,168.0,0.99490,3.15,0.46,9.6,5
4895,6.5,0.24,0.19,1.2,0.041,30.0,111.0,0.99254,2.99,0.46,9.4,6
4896,5.5,0.29,0.30,1.1,0.022,20.0,110.0,0.98869,3.34,0.38,12.8,7


### 実行処理のロギング

PandasDatasetのインスタンス`dataset`を以下の通り作成する。  
特に`source`によりデータソースのURLを指定するが、HTTP URLだけでなく、S3 URIなどの指定も可能である。

In [6]:
dataset = mlflow.data.from_pandas(
      df
    , source=dataset_source_url
    , name="wine quality - white"
    , targets="quality"
)
dataset

/home/user01/.local/lib/python3.10/site-packages/mlflow/data/digest_utils.py:26: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  string_columns = trimmed_df.columns[(df.applymap(type) == str).all(0)]


実行処理をロギングする。

In [7]:
run_name = "Dataset Track Trial"

with mlflow.start_run(run_name=run_name) as run:
    mlflow.log_input(dataset, context="train/test")

/home/user01/.local/lib/python3.10/site-packages/mlflow/types/utils.py:393: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details.
  warnings.warn(


ロギングが終わったら、MLFLow UIより今回のエクスペリメントの結果を確認する。  
ここでは特にDatasetsの扱いについて確認をしてほしい。

### 実行処理に関する出力

In [8]:
# 実行処理の情報の取得
logged_run = mlflow.get_run(run.info.run_id)
logged_run

<Run: data=<RunData: metrics={}, params={}, tags={'mlflow.runName': 'Dataset Track Trial',
 'mlflow.source.name': '/home/user01/.local/lib/python3.10/site-packages/ipykernel_launcher.py',
 'mlflow.source.type': 'LOCAL',
 'mlflow.user': 'user01'}>, info=<RunInfo: artifact_uri='mlflow-artifacts:/902688651372085152/5f845e4899564f16b795080a012d96c5/artifacts', end_time=1713942087754, experiment_id='902688651372085152', lifecycle_stage='active', run_id='5f845e4899564f16b795080a012d96c5', run_name='Dataset Track Trial', run_uuid='5f845e4899564f16b795080a012d96c5', start_time=1713942087686, status='FINISHED', user_id='user01'>, inputs=<RunInputs: dataset_inputs=[<DatasetInput: dataset=<Dataset: digest='2a1e42c4', name='wine quality - white', profile='{"num_rows": 4898, "num_elements": 58776}', schema=('{"mlflow_colspec": [{"type": "double", "name": "fixed acidity", "required": '
 'true}, {"type": "double", "name": "volatile acidity", "required": true}, '
 '{"type": "double", "name": "citric a

In [9]:
# Datasetオブジェクトの取得
logged_dataset = logged_run.inputs.dataset_inputs[0].dataset
logged_dataset

<Dataset: digest='2a1e42c4', name='wine quality - white', profile='{"num_rows": 4898, "num_elements": 58776}', schema=('{"mlflow_colspec": [{"type": "double", "name": "fixed acidity", "required": '
 'true}, {"type": "double", "name": "volatile acidity", "required": true}, '
 '{"type": "double", "name": "citric acid", "required": true}, {"type": '
 '"double", "name": "residual sugar", "required": true}, {"type": "double", '
 '"name": "chlorides", "required": true}, {"type": "double", "name": "free '
 'sulfur dioxide", "required": true}, {"type": "double", "name": "total sulfur '
 'dioxide", "required": true}, {"type": "double", "name": "density", '
 '"required": true}, {"type": "double", "name": "pH", "required": true}, '
 '{"type": "double", "name": "sulphates", "required": true}, {"type": '
 '"double", "name": "alcohol", "required": true}, {"type": "long", "name": '
 '"quality", "required": true}]}'), source=('{"url": '
 '"https://raw.githubusercontent.com/mlflow/mlflow/master/tests/d

In [10]:
# 記録されたDatasetの情報をいくつか確認する
print(f"Dataset name: {logged_dataset.name}")
print(f"Dataset digest: {logged_dataset.digest}")
print(f"Dataset profile: {logged_dataset.profile}")
print(f"Dataset schema: {logged_dataset.schema}")

Dataset name: wine quality - white
Dataset digest: 2a1e42c4
Dataset profile: {"num_rows": 4898, "num_elements": 58776}
Dataset schema: {"mlflow_colspec": [{"type": "double", "name": "fixed acidity", "required": true}, {"type": "double", "name": "volatile acidity", "required": true}, {"type": "double", "name": "citric acid", "required": true}, {"type": "double", "name": "residual sugar", "required": true}, {"type": "double", "name": "chlorides", "required": true}, {"type": "double", "name": "free sulfur dioxide", "required": true}, {"type": "double", "name": "total sulfur dioxide", "required": true}, {"type": "double", "name": "density", "required": true}, {"type": "double", "name": "pH", "required": true}, {"type": "double", "name": "sulphates", "required": true}, {"type": "double", "name": "alcohol", "required": true}, {"type": "long", "name": "quality", "required": true}]}
